In [ ]:
from selenium import webdriver
from tqdm import tqdm_notebook as tqdmn
import pandas as pd
import folium

#lst = ['20200927_123015.csv', '20200927_135221.csv','20200927_145701.csv','20200927_161843.csv','20200928_074659.csv','20200928_081904.csv','20200928_083657.csv','20200928_090044.csv','20200928_090822.csv','20200928_101857.csv','20200928_102907.csv','20200928_114457.csv','20200928_115312.csv']
lst = [
"C:\\Users\\German\\Desktop\\Курсовая работа\\Алгоритмы\\Датасеты\\Almaty_Success\\20210124_220707.csv",
"C:\\Users\\German\\Desktop\\Курсовая работа\\Алгоритмы\\Датасеты\\Almaty_Success\\20210126_000100.csv",
"C:\\Users\\German\\Desktop\\Курсовая работа\\Алгоритмы\\Датасеты\\Almaty_Success\\20210126_234730.csv",
"C:\\Users\\German\\Desktop\\Курсовая работа\\Алгоритмы\\Датасеты\\Almaty_Success\\20210127_221752.csv",
"C:\\Users\\German\\Desktop\\Курсовая работа\\Алгоритмы\\Датасеты\\Almaty_Success\\20210130_000350.csv",
"C:\\Users\\German\\Desktop\\Курсовая работа\\Алгоритмы\\Датасеты\\Almaty_Success\\20210130_233044.csv",
"C:\\Users\\German\\Desktop\\Курсовая работа\\Алгоритмы\\Датасеты\\Almaty_Success\\20210131_225325.csv",
"C:\\Users\\German\\Desktop\\Курсовая работа\\Алгоритмы\\Датасеты\\Almaty_Success\\20210204_233215.csv",
"C:\\Users\\German\\Desktop\\Курсовая работа\\Алгоритмы\\Датасеты\\Almaty_Success\\20210206_000436.csv",
"C:\\Users\\German\\Desktop\\Курсовая работа\\Алгоритмы\\Датасеты\\Almaty_Success\\20210206_233212.csv",
"C:\\Users\\German\\Desktop\\Курсовая работа\\Алгоритмы\\Датасеты\\Almaty_Success\\20210207_230546.csv",
"C:\\Users\\German\\Desktop\\Курсовая работа\\Алгоритмы\\Датасеты\\Almaty_Success\\20210208_222901.csv",
"C:\\Users\\German\\Desktop\\Курсовая работа\\Алгоритмы\\Датасеты\\Almaty_Success\\20210209_235544.csv",
"C:\\Users\\German\\Desktop\\Курсовая работа\\Алгоритмы\\Датасеты\\Almaty_Success\\20210210_222134.csv",
"C:\\Users\\German\\Desktop\\Курсовая работа\\Алгоритмы\\Датасеты\\Almaty_Success\\20210211_214900.csv",
"C:\\Users\\German\\Desktop\\Курсовая работа\\Алгоритмы\\Датасеты\\Almaty_Success\\20210212_213212.csv",
"C:\\Users\\German\\Desktop\\Курсовая работа\\Алгоритмы\\Датасеты\\Almaty_Success\\20210214_002758.csv",
"C:\\Users\\German\\Desktop\\Курсовая работа\\Алгоритмы\\Датасеты\\Almaty_Success\\20210215_020241.csv",
"C:\\Users\\German\\Desktop\\Курсовая работа\\Алгоритмы\\Датасеты\\Almaty_Success\\20210216_001600.csv",
"C:\\Users\\German\\Desktop\\Курсовая работа\\Алгоритмы\\Датасеты\\Almaty_Success\\20210217_003125.csv",
"C:\\Users\\German\\Desktop\\Курсовая работа\\Алгоритмы\\Датасеты\\Almaty_Success\\20210217_220604.csv",
"C:\\Users\\German\\Desktop\\Курсовая работа\\Алгоритмы\\Датасеты\\Almaty_Success\\20210218_220459.csv",
"C:\\Users\\German\\Desktop\\Курсовая работа\\Алгоритмы\\Датасеты\\Almaty_Success\\20210219_233905.csv",
"C:\\Users\\German\\Desktop\\Курсовая работа\\Алгоритмы\\Датасеты\\Almaty_Success\\20210220_235130.csv",
"C:\\Users\\German\\Desktop\\Курсовая работа\\Алгоритмы\\Датасеты\\Almaty_Success\\20210221_235423.csv",
"C:\\Users\\German\\Desktop\\Курсовая работа\\Алгоритмы\\Датасеты\\Almaty_Success\\20210222_214759.csv"]
for i in range(len(lst)):
    file_name = lst[i]
    acnc = pd.read_csv(file_name, sep= "\t", keep_default_na=False)
    mel = acnc[
        ['OrderID', 'Weight', 'PickupAddress', 'DeliveryAddress']].copy()
    mel['Url_depo'] = ['https://www.google.com/maps/search/' + i for i in mel['PickupAddress'] ]
    mel['Url_clients'] = ['https://www.google.com/maps/search/' + i for i in mel['DeliveryAddress'] ]
    Url_With_Coordinates_depo = []
    Url_With_Coordinates_clients = []

    option = webdriver.ChromeOptions()
    prefs = {'profile.default_content_setting_values': {}}
    option.add_experimental_option('prefs', prefs)

    driver = webdriver.Chrome("chromedriver.exe", options=option)

    for url in tqdmn(mel.Url_depo, leave=False):
        driver.get(url)
        Url_With_Coordinates_depo.append(driver.find_element_by_css_selector('meta[itemprop=image]').get_attribute('content'))

    for url in tqdmn(mel.Url_clients, leave=False):
        driver.get(url)
        Url_With_Coordinates_clients.append(driver.find_element_by_css_selector('meta[itemprop=image]').get_attribute('content'))

    driver.close()

    mel['Url_With_Coordinates_depo'] = Url_With_Coordinates_depo
    mel['Url_With_Coordinates_clients'] = Url_With_Coordinates_clients
    mel = mel[mel.Url_With_Coordinates_clients.str.contains('&zoom=')].copy()
    
    mel['x_depo'] = [ url.split('?center=')[1].split('&zoom=')[0].split('%2C')[0] for url in mel['Url_With_Coordinates_depo'] ]
    mel['y_depo'] = [url.split('?center=')[1].split('&zoom=')[0].split('%2C')[1] for url in mel['Url_With_Coordinates_depo'] ]
    mel['x_client'] = [ url.split('?center=')[1].split('&zoom=')[0].split('%2C')[0] for url in mel['Url_With_Coordinates_clients'] ]
    mel['y_client'] = [url.split('?center=')[1].split('&zoom=')[0].split('%2C')[1] for url in mel['Url_With_Coordinates_clients'] ]

    mel = mel.drop('OrderID', 1)
    mel = mel.drop('PickupAddress', 1)
    mel = mel.drop('DeliveryAddress', 1)
    mel = mel.drop('Url_depo', 1)
    mel = mel.drop('Url_clients', 1)
    mel = mel.drop('Url_With_Coordinates_depo', 1)
    mel = mel.drop('Url_With_Coordinates_clients', 1)

    mel.to_csv(file_name, sep='\t')

<ipython-input-1-ab41ff0bf81b>:48: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe", options=option)
<ipython-input-1-ab41ff0bf81b>:50: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for url in tqdmn(mel.Url_depo, leave=False):


  0%|          | 0/316 [00:00<?, ?it/s]

<ipython-input-1-ab41ff0bf81b>:52: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  Url_With_Coordinates_depo.append(driver.find_element_by_css_selector('meta[itemprop=image]').get_attribute('content'))
<ipython-input-1-ab41ff0bf81b>:54: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for url in tqdmn(mel.Url_clients, leave=False):


  0%|          | 0/316 [00:00<?, ?it/s]

<ipython-input-1-ab41ff0bf81b>:56: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  Url_With_Coordinates_clients.append(driver.find_element_by_css_selector('meta[itemprop=image]').get_attribute('content'))


...

  0%|          | 0/316 [00:00<?, ?it/s]

  0%|          | 0/316 [00:00<?, ?it/s]

  0%|          | 0/316 [00:00<?, ?it/s]

  0%|          | 0/316 [00:00<?, ?it/s]

  0%|          | 0/316 [00:00<?, ?it/s]

In [ ]:
# #pd.set_option('display.max_rows', None)
 
# # Сброс ограничений на число столбцов
# pd.set_option('display.max_columns', None)
 
# # Сброс ограничений на количество символов в записи
# pd.set_option('display.max_colwidth', None)
# mel